In [2]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count

### Determining food names and tables

In [6]:
root = "../raw/it.openfoodfacts.org"
foods = [food for food in os.listdir(root) if food[0] != "."]

### Merging csv files

In [59]:
def parse_food(food):
    root = "../raw/it.openfoodfacts.org"
    file_path = "{path}/{name}/Informazioni nutrizionali.csv".format(
        path=root, name=food)
    try:
        df = pd.read_csv(
            file_path,
            usecols=[
                "Informazioni nutrizionali", "Come vendutoper 100 g / 100 ml"
            ],
            index_col="Informazioni nutrizionali")
    except Exception as e:
        return None
    if (df == "?").any().any():
        return None
    for index, value in df.iterrows():
        value = value["Come vendutoper 100 g / 100 ml"]
        if isinstance(value, str):
            unit = ""
            value = value.replace(".", "").replace(",", ".").replace("<", "")
            if "mg" in value:
                value = float(value.split("mg")[0])*1e-3
                unit = "g"
            elif "µg" in value:
                value = float(value.split("µg")[0])*1e-6
                unit = "g"
            elif "g" in value:
                value = float(value.split("g")[0])
                unit = "g"
            elif re.match("-?[\dA-Z]", value):
                pass
            else:
                raise ValueError(str((value, index)))
            df.loc[index] = value
            if unit:
                df = df.rename(
                    index={
                        index:
                        "{index} | {unit}".format(index=index, unit=unit).lower()
                        .strip().replace("  ", " ")
                    })

    df = df.transpose()
    with open("{path}/{name}/metadata.json".format(path=root, name=food),
              "r") as f:
        for key, value in json.load(f).items():
            if value:
                df[key] = value
    df.index = df["name"]
    del df.columns.name
    return df

In [61]:
from IPython.display import display
with Pool(cpu_count()) as p:
    open_food_facts = pd.concat(list(tqdm(p.imap(parse_food, foods), total=len(foods))))
open_food_facts.index.name = "name"
open_food_facts.to_csv("../csv/openfoodsfacts.csv")

In [62]:
open_food_facts

,- acidi grassi monoinsaturi | g,- acidi grassi omega 6 | g,- acidi grassi polinsaturi | g,- acidi grassi saturi | g,- acidi grassi trans | g,- amido | g,- colesterolo | g,- lattosio | g,- polialcoli/polioli (alcoli degli zuccheri) | g,- zuccheri | g,...,vitamina b2 (riboflavina) | g,vitamina b3 / vitamina pp (niacina) | g,vitamina b5 (acido pantotenico) | g,vitamina b6 (piridoxina) | g,vitamina b8/b7/h/i (biotina) | g,vitamina b9 (acido folico) | g,vitamina c (acido ascorbico) | g,vitamina d (colecalciferolo) | g,vitamina e (alfa-tocoferolo) | g,zinco | g
name,,,,,,,,,,,,,,,,,,,,,
Ricotta — Conad — 250 g,NaN,NaN,NaN,9.6,NaN,NaN,NaN,NaN,NaN,3.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yaourt — Sterzing-Vipiteno — 125 g e,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Piccolinis Pomodoro Mozzarella — Buitoni — 270 g, 9 pizza de 30 g",NaN,NaN,NaN,3.7,NaN,NaN,NaN,NaN,NaN,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Latte parzialmente scremato UHT — Verso Natura — 1 L,NaN,NaN,NaN,1.1,NaN,NaN,NaN,NaN,NaN,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bevanda a base di avena — Verso Natura — 1 L,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,7.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rigoni Nocciolata crema di cacao e nocciole Prodotto biologico — Rigoni di Asiago — 350 g,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Crostata ala ciliegia — Santangelo — 350 g,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN,NaN,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fondente con pezzi di cocco — Lindt,NaN,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gelato base Mandorla — VALSOIA — 50g,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
np.mean(np.mean(pd.isna(open_food_facts)))

0.7461003211500229